In [1]:
%pip install yfinance pandas
import yfinance as yf
import pandas as pd

  Using cached yfinance-0.2.18-py2.py3-none-any.whl (60 kB)
  Using cached frozendict-2.3.8-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (101 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Using cached lxml-4.9.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (6.6 MB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
Note: you may need to restart the kernel to use updated packages.


# First let's play around with yfinance to suss out how we'll source the data

In [54]:
# https://github.com/ranaroussi/yfinance
goog = yf.Ticker('GOOG')
goog.info['previousClose']

105.98

In [47]:
goog = yf.download('GOOG', start='2022-01-01', end='2023-05-03')
goog

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-03,144.475494,145.550003,143.502502,145.074493,145.074493,25214000
2022-01-04,145.550507,146.610001,143.816147,144.416504,144.416504,22928000
2022-01-05,144.181000,144.298004,137.523499,137.653503,137.653503,49642000
2022-01-06,137.497498,139.686005,136.763504,137.550995,137.550995,29050000
2022-01-07,137.904999,138.254745,135.789001,137.004501,137.004501,19408000
...,...,...,...,...,...,...
2023-04-26,105.559998,107.019997,103.269997,104.449997,104.449997,37068200
2023-04-27,105.230003,109.150002,104.419998,108.370003,108.370003,38235200
2023-04-28,107.800003,108.290001,106.040001,108.220001,108.220001,23957900


In [42]:
goog = yf.Ticker('^SPX').history(period='1d')
goog['Ticker'] = '^SPX'
goog

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker
Date,,,,,,,,
2023-05-04 00:00:00-04:00,4082.550049,4082.610107,4049.080078,4059.580078,808729118,0.0,0.0,^SPX


In [48]:
condition = goog.index.to_series().eq('2023-02-03') #could also .between('2023-02-01', '2023-02-03')
goog[condition]

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-02-03,103.510002,108.019997,103.300003,105.220001,105.220001,36823400


# Let's compose a simple market data tabular dataset for a set tickers that have an assumed price relationship

In [43]:
tickers = ['^DJI', '^NDX', '^VIX', '^SPX', 'XLK', 'QQQ', 'GOOG', 'AAPL', 'MSFT', 'META', 
           'NFLX', 'AXP', 'AMGN', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM',
           'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'NKE', 'PG', 'TRV', 'UNH',
           'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']
columns = ['Open', 'High', 'Low', 'Close', 'Volume']
dfs = []
for ticker in tickers:
    print('Processing %s' % ticker)
    df = yf.Ticker(ticker).history(period='25y')
    df = df[columns]
    df['Ticker'] = ticker
    #df = df.add_prefix(ticker+'_')
    dfs.append(df)

Processing ^DJI
Processing ^NDX
Processing ^VIX
Processing ^SPX
Processing XLK
Processing QQQ
Processing GOOG
Processing AAPL
Processing MSFT
Processing META
Processing NFLX
Processing AXP
Processing AMGN
Processing BA
Processing CAT
Processing CSCO
Processing CVX
Processing GS
Processing HD
Processing HON
Processing IBM
Processing INTC
Processing JNJ
Processing KO
Processing JPM
Processing MCD
Processing MMM
Processing MRK
Processing NKE
Processing PG
Processing TRV
Processing UNH
Processing CRM
Processing VZ
Processing V
Processing WBA
Processing WMT
Processing DIS
Processing DOW


In [44]:
#from functools import reduce
#market_data = reduce(lambda df1,df2: pd.merge(df1,df2,on='Date'), dfs)
market_data = pd.concat(dfs)
market_data.reset_index(inplace=True)
market_data

,Date,Open,High,Low,Close,Volume,Ticker
0,1998-05-04 00:00:00-04:00,9145.330078,9261.910156,9145.330078,9192.660156,50980000,^DJI
1,1998-05-05 00:00:00-04:00,9192.660156,9198.139648,9104.469727,9147.570312,42750000,^DJI
2,1998-05-06 00:00:00-04:00,9146.190430,9180.200195,9049.919922,9054.650391,45340000,^DJI
3,1998-05-07 00:00:00-04:00,9053.160156,9055.650391,8965.219727,8976.679688,52340000,^DJI
4,1998-05-08 00:00:00-04:00,8978.169922,9095.750000,8971.200195,9055.150391,51560000,^DJI
...,...,...,...,...,...,...,...
223085,2023-04-28 00:00:00-04:00,53.240002,54.910000,53.169998,54.400002,6014400,DOW
223086,2023-05-01 00:00:00-04:00,54.459999,54.619999,53.825001,54.110001,4013200,DOW
223087,2023-05-02 00:00:00-04:00,53.500000,53.889999,52.580002,53.779999,4206000,DOW
223088,2023-05-03 00:00:00-04:00,53.750000,54.169998,52.910000,53.000000,3342000,DOW


In [45]:
def label_strategy (row):
   if row['Close'] - row['Open'] >= row['Open'] * .04 :
      return 'Sell'
   if row['Close'] < row['Open'] :
      return 'Buy'
   return 'Hold'

market_data['strategy'] = market_data.apply(lambda row: label_strategy(row), axis=1)

In [46]:
market_data.to_csv('market_data.csv', index=False)

# Experimental

In [ ]:
import requests

access_key = '2b607874eed4b703985d61f132488406'
symbols = 'MSFT'

api_result = requests.get('http://api.marketstack.com/v1/eod?access_key=%s&symbols=%s' % (access_key, symbols))

api_response = api_result.json()